In [1]:
# imports
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
import requests
import csv

In [3]:
import time
import json

In [4]:
# breaking bad reddit url
url = 'https://www.reddit.com/r/breakingbad/.json'

In [5]:
# set headers 
headers = {'User-Agent': 'Mozilla/5.0'}

# check response of url
page = requests.get(url, headers=headers)
print(page.status_code)

200


In [6]:
# convert page to json 
the_json = page.json()

In [7]:
# check keys
the_json.keys()

dict_keys(['kind', 'data'])

In [8]:
# find last user
the_json['data']['after']

't3_cauxt1'

In [9]:
# confirm above
[post['data']['name'] for post in the_json['data']['children']]

['t3_cb056z',
 't3_cap5x8',
 't3_cax09s',
 't3_cb3whu',
 't3_cb47lz',
 't3_cb3txf',
 't3_canr4o',
 't3_cb3dgi',
 't3_cb43wv',
 't3_cb3vht',
 't3_cau3nb',
 't3_cb46ro',
 't3_cb3m49',
 't3_canl2f',
 't3_cb0r0i',
 't3_cak8oh',
 't3_cac4ll',
 't3_caua60',
 't3_cazjqb',
 't3_caomae',
 't3_caosz4',
 't3_calb8h',
 't3_caqj1o',
 't3_catjis',
 't3_cauxt1']

In [10]:
# set params to last post 
params = {'after':'t3_cauxt1' }

In [11]:
# final url response check
requests.get(url, params=params, headers=headers)

<Response [200]>

In [12]:
# loop to scrape posts

posts = []
after = None

for i in range(30):
    print(i)
    
    if after == None:
        params = {}
    else:
        params = {'after':after}
    url = 'https://www.reddit.com/r/breakingbad/.json'
    res = requests.get(url, params=params, headers=headers)
    
    if res.status_code == 200:
        the_json = res.json()
        posts.extend(the_json['data']['children'])
        after = the_json['data']['after']
    else:
        print(res.status_code)
        break
        
    time.sleep(3)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [13]:
# check for duplicates
len(set([p['data']['name'] for p in posts]))

750

In [14]:
# save to a json file to preserve all information scraped
# with open('../json_files/breakingbad_tuesday.json', 'w') as f:
#     json.dump(posts , f)

In [8]:
# open saved json file for editing
# with open(r"../json_files/breakingbad_tuesday.json", "r") as read_file:
#     posts = json.load(read_file)

In [15]:
# loop to extract desired data from posts 

post_info = []

i = 0
for post in posts:
    info = {}
    
    info['title'] = posts[i]['data']['title']
    info['selftext'] = posts[i]['data']['selftext']
    info['author'] = posts[i]['data']['author']
    info['ups'] = posts[i]['data']['ups']
    info['downs'] = posts[i]['data']['downs']
    info['score'] = posts[i]['data']['score']
    info['num_comments'] = posts[i]['data']['num_comments']
    info['created_utc'] = posts[i]['data']['created_utc']
    post_info.append(info)
    
    i += 1

# create dataframe
post_info_df = pd.DataFrame(post_info)


In [16]:
import datetime

In [17]:
# convert to proper datetime
post_info_df['created_utc'] = pd.to_datetime(post_info_df['created_utc'], unit='s')

In [18]:
post_info_df.head()

,author,created_utc,downs,num_comments,score,selftext,title,ups
745,Ronin0206,2019-06-07 17:22:00,0,25,21,Nust finished the show and it may as well be t...,Just finished the show!,21
746,JK-Network123,2019-06-07 22:49:46,0,11,6,Just curious but say Walter White aka Heisenbe...,How infamous would Walter White be if he was R...,6
747,SergeantBubbles7,2019-06-07 23:28:37,0,1,4,Do they really live sparsely in reservations o...,Is Breaking Bad’s representation of Native Ame...,4
748,Nyxeme,2019-06-07 10:20:06,0,9,55,Yesterday my girlfriend and I went out for sus...,I saw Gus Fring yesterday!,55
749,chad2215,2019-06-07 19:10:40,0,57,10,What’s this guys problem man? So greedy ... co...,Walter is sick in the head man....,10


In [19]:
# check keys
post_info_df.keys()

Index(['author', 'created_utc', 'downs', 'num_comments', 'score', 'selftext',
       'title', 'ups'],
      dtype='object')

In [20]:
# create csv from dataframe
# post_info_df.to_csv('../df_csv/breakingbad_tuesday.csv', index=True)